In [4]:
!pip install keras_tuner
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import keras_tuner as kt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

import pickle

import warnings
warnings.filterwarnings('ignore')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.2 MB/s eta 0:00:00


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
test_dataset = pd.read_csv("/content/drive/MyDrive/AI_workout_assistant/test.csv")
test_dataset.columns
train_dataset = pd.read_csv("/content/drive/MyDrive/AI_workout_assistant/train.csv")

In [7]:
test_dataset.shape
test_dataset.info()
test_dataset.describe()

train_dataset.shape
train_dataset.info()
train_dataset.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 710 entries, 0 to 709
Data columns (total 69 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   label               710 non-null    object 
 1   nose_x              710 non-null    float64
 2   nose_y              710 non-null    float64
 3   nose_z              710 non-null    float64
 4   nose_v              710 non-null    float64
 5   left_shoulder_x     710 non-null    float64
 6   left_shoulder_y     710 non-null    float64
 7   left_shoulder_z     710 non-null    float64
 8   left_shoulder_v     710 non-null    float64
 9   right_shoulder_x    710 non-null    float64
 10  right_shoulder_y    710 non-null    float64
 11  right_shoulder_z    710 non-null    float64
 12  right_shoulder_v    710 non-null    float64
 13  left_elbow_x        710 non-null    float64
 14  left_elbow_y        710 non-null    float64
 15  left_elbow_z        710 non-null    float64
 16  left_elb

,nose_x,nose_y,nose_z,nose_v,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,right_shoulder_y,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
count,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,...,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000
mean,0.510215,0.581268,-0.015778,0.999339,0.497311,0.507337,0.007344,0.996716,0.517837,0.515554,...,0.059029,0.799373,0.495743,0.725944,0.006097,0.760909,0.474075,0.727088,-0.007818,0.801929
std,0.271763,0.110770,0.069960,0.001282,0.212051,0.116594,0.180132,0.005807,0.211902,0.114373,...,0.145910,0.175780,0.340344,0.091236,0.167639,0.203684,0.339669,0.091583,0.165447,0.177213
min,-0.020205,0.282129,-0.628718,0.983588,0.072622,0.258629,-0.504613,0.959863,0.142915,0.254793,...,-0.297123,0.263962,0.008636,0.436971,-0.589263,0.164304,0.007629,0.322390,-0.374601,0.257708
25%,0.217230,0.513137,-0.030574,0.999396,0.272365,0.433988,-0.162033,0.996988,0.301728,0.442456,...,-0.064933,0.649804,0.144528,0.657492,-0.121383,0.664713,0.117348,0.663741,-0.147351,0.653375
50%,0.522769,0.578837,-0.009271,0.999817,0.506357,0.487884,0.078979,0.999177,0.511510,0.495600,...,0.066489,0.900963,0.404329,0.713178,0.028276,0.797713,0.393740,0.714858,0.005031,0.890545
75%,0.792730,0.655644,0.013992,0.999919,0.712670,0.583688,0.174769,0.999695,0.734849,0.593504,...,0.149279,0.956642,0.853447,0.816601,0.102518,0.941481,0.834829,0.817925,0.091140,0.941260
max,0.926463,0.821666,0.229125,0.999995,0.812100,0.755157,0.337118,0.999962,0.833827,0.753441,...,0.780961,0.999172,1.168851,0.963118,0.779786,0.986748,1.144594,0.956528,0.781242,0.998428


In [8]:
train_dataset.head()
test_dataset.head()

,label,nose_x,nose_y,nose_z,nose_v,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
0,C,0.241842,0.435841,-0.002772,0.999881,0.274594,0.341100,-0.188556,0.999743,0.296270,...,0.174394,0.529667,0.801527,0.555134,-0.070554,0.776038,0.794738,0.537268,0.102899,0.440595
1,C,0.237662,0.440657,-0.003768,0.999896,0.274228,0.339355,-0.189618,0.999759,0.296200,...,0.184522,0.553936,0.803268,0.557891,-0.061299,0.799590,0.795147,0.533265,0.113687,0.479885
2,C,0.237591,0.441506,-0.003623,0.999899,0.274257,0.340742,-0.190390,0.999762,0.295690,...,0.187280,0.553449,0.803358,0.558061,-0.053722,0.798296,0.797029,0.534547,0.120511,0.478705
3,C,0.238127,0.441322,-0.000322,0.999902,0.274333,0.339990,-0.190143,0.999760,0.295731,...,0.178452,0.561389,0.803932,0.554631,-0.059433,0.804831,0.799963,0.532089,0.107540,0.488323
4,C,0.238066,0.440833,0.003989,0.999907,0.274171,0.343735,-0.187406,0.999760,0.295011,...,0.172571,0.567620,0.802851,0.554338,-0.064519,0.809826,0.799751,0.530627,0.102761,0.495874


In [9]:
train_dataset["label"].value_counts()

,count
label,
C,9904
L,9546
H,9070


In [10]:
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_ELBOW",
    "RIGHT_ELBOW",
    "LEFT_WRIST",
    "RIGHT_WRIST",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
    "LEFT_HEEL",
    "RIGHT_HEEL",
    "LEFT_FOOT_INDEX",
    "RIGHT_FOOT_INDEX",
]

HEADERS = ["label"]

for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z"]

In [11]:
def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")

    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data

def remove_duplicate_rows(dataset_path: str):
    '''
    Remove duplicated data from the dataset then save it to another files
    '''

    df = pd.read_csv(dataset_path)
    df.drop_duplicates(keep="first", inplace=True)
    df.to_csv(f"cleaned_train.csv", sep=',', encoding='utf-8', index=False)


def round_up_metric_results(results) -> list:
    '''Round up metrics results such as precision score, recall score, ...'''
    return list(map(lambda el: round(el, 3), results))


df = describe_dataset("/content/drive/MyDrive/AI_workout_assistant/train.csv")

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v', 'left_heel_x', 'left_heel_y', 'left_heel_z', 'left_heel_v', 'right_heel_x', 'right_heel_y', 'right_heel_z', 'right_heel_v', 'left_foot_index_x', 'lef

2. Preprocess Dataset

In [12]:
df = pd.read_csv("/content/drive/MyDrive/AI_workout_assistant/train.csv")

df.loc[df["label"] == "C", "label"] = 0
df.loc[df["label"] == "H", "label"] = 1
df.loc[df["label"] == "L", "label"] = 2

print(f'Number of rows: {df.shape[0]} \nNumber of columns: {df.shape[1]}\n')
print(f"Labels: \n{df['label'].value_counts()}\n")

Number of rows: 28520 
Number of columns: 69

Labels: 
label
0    9904
2    9546
1    9070
Name: count, dtype: int64



In [13]:
# Standard Scaling of features
# Dump input scaler
with open("/content/drive/MyDrive/AI_workout_assistant/input_scaler.pkl", "rb") as f2:
    sc = pickle.load(f2)

x = df.drop("label", axis = 1)
x = pd.DataFrame(sc.transform(x))

y = df["label"]

# # Converting prediction to categorical
y_cat = to_categorical(y)

print(df.drop("label", axis=1).columns)

Index(['nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x',
       'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v',
       'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z',
       'right_shoulder_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z',
       'left_elbow_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z',
       'right_elbow_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z',
       'left_wrist_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z',
       'right_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v',
       'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v',
       'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v',
       'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v',
       'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v',
       'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v',
       'left_heel_x', 'left_heel_y', 'left_heel_z', 'left_heel_v',
       'ri

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2, random_state=1234)

3. Build Model

In [15]:
stop_early = EarlyStopping(monitor='loss', patience=3)

# Final Results
final_models = {}

In [16]:
def describe_model(model):
    '''
    Describe Model architecture
    '''
    print(f"Describe models architecture")
    for i, layer in enumerate(model.layers):
        number_of_units = layer.units if hasattr(layer, 'units') else 0

        if hasattr(layer, "activation"):
            print(f"Layer-{i + 1}: {number_of_units} units, func: ", layer.activation)
        else:
            print(f"Layer-{i + 1}: {number_of_units} units, func: None")


def get_best_model(tuner):
    '''
    Describe and return the best model found from keras tuner
    '''
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_model = tuner.hypermodel.build(best_hps)

    describe_model(best_model)

    print("\nOther params:")
    ignore_params = ["tuner", "activation", "layer"]
    for param, value in best_hps.values.items():
        if not any(word in param for word in ignore_params):
            print(f"{param}: {value}")

    return best_model

In [17]:
def model_3l_builder(hp):
    model = Sequential()
    model.add(Dense(68, input_dim = 68, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(3, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])

    return model

In [18]:
tuner_3l = kt.Hyperband(
    model_3l_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo'
)
tuner_3l.search(x_train, y_train, epochs=10, callbacks=[stop_early])

Trial 30 Complete [00h 00m 18s]
accuracy: 0.997808575630188

Best accuracy So Far: 0.9993425607681274
Total elapsed time: 00h 04m 11s


In [19]:
model_3l = get_best_model(tuner_3l)
model_3l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 68 units, func:  <function relu at 0x7cf30dbff2e0>
Layer-2: 352 units, func:  <function tanh at 0x7cf2b715e830>
Layer-3: 3 units, func:  <function softmax at 0x7cf2b715e440>

Other params:
learning_rate: 0.001
Epoch 1/100
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.9647 - loss: 0.0996 - val_accuracy: 0.9946 - val_loss: 0.0108
Epoch 2/100
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9960 - loss: 0.0146 - val_accuracy: 0.9989 - val_loss: 0.0027
Epoch 3/100
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.9970 - loss: 0.0109 - val_accuracy: 0.9989 - val_loss: 0.0022
Epoch 4/100
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9977 - loss: 0.0087 - val_accuracy: 0.9993 - val_loss: 0.0026
Epoch 5/100
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9979 - loss: 0.0088 - val_accuracy: 0.9991 - val_loss: 0.0023
Epoch 6/100
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9986 - loss: 0.0067 - val_ac

In [20]:
final_models["3_layers"] = model_3l

In [21]:
def model_5l_builder(hp):
    model = Sequential()
    model.add(Dense(68, input_dim = 68, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(3, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])

    return model

In [22]:
tuner_5l = kt.Hyperband(
    model_5l_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_1'
)
tuner_5l.search(
    x_train,
    y_train,
    epochs=10,
    callbacks=[EarlyStopping(monitor='val_loss', patience=5)]
)

Trial 30 Complete [00h 00m 28s]
accuracy: 0.9989480972290039

Best accuracy So Far: 0.9992548823356628
Total elapsed time: 00h 05m 59s


In [23]:
model_5l = get_best_model(tuner_5l)
model_5l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 68 units, func:  <function relu at 0x7cf30dbff2e0>
Layer-2: 128 units, func:  <function tanh at 0x7cf2b715e830>
Layer-3: 320 units, func:  <function tanh at 0x7cf2b715e830>
Layer-4: 320 units, func:  <function tanh at 0x7cf2b715e830>
Layer-5: 3 units, func:  <function softmax at 0x7cf2b715e440>

Other params:
learning_rate: 0.0001
Epoch 1/100
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.9393 - loss: 0.1936 - val_accuracy: 0.9981 - val_loss: 0.0105
Epoch 2/100
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9980 - loss: 0.0082 - val_accuracy: 0.9991 - val_loss: 0.0030
Epoch 3/100
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9981 - loss: 0.0081 - val_accuracy: 0.9996 - val_loss: 0.0021


In [24]:
final_models["5_layers"] = model_5l

In [25]:
def model_7lD_builder(hp):
    model = Sequential()
    model.add(Dense(68, input_dim = 68, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_dropout_1 = hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)
    hp_dropout_2 = hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_1))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_2))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(3, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])

    return model

In [26]:
tuner_7lD = kt.Hyperband(
    model_7lD_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_4'
)
tuner_7lD.search(x_train, y_train, epochs=10, callbacks=[stop_early])

Trial 30 Complete [00h 00m 12s]
accuracy: 0.9982468485832214

Best accuracy So Far: 0.9992548823356628
Total elapsed time: 00h 05m 22s


In [27]:
model_7lD = get_best_model(tuner_7lD)
model_7lD.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 68 units, func:  <function relu at 0x7cf30dbff2e0>
Layer-2: 192 units, func:  <function tanh at 0x7cf2b715e830>
Layer-3: 0 units, func: None
Layer-4: 128 units, func:  <function tanh at 0x7cf2b715e830>
Layer-5: 0 units, func: None
Layer-6: 128 units, func:  <function tanh at 0x7cf2b715e830>
Layer-7: 3 units, func:  <function softmax at 0x7cf2b715e440>

Other params:
dropout_1: 0.1
dropout_2: 0.1
learning_rate: 0.0001
Epoch 1/100
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.8995 - loss: 0.2939 - val_accuracy: 0.9982 - val_loss: 0.0082
Epoch 2/100
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9976 - loss: 0.0113 - val_accuracy: 0.9986 - val_loss: 0.0049
Epoch 3/100
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9975 - loss: 0.0087 - val_accuracy: 0.9991 - val_loss: 0.0030


In [28]:
final_models["7_layers_with_dropout"] = model_7lD

In [29]:
def model_7l_builder(hp):
    model = Sequential()
    model.add(Dense(68, input_dim = 68, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_layer_4 = hp.Int('layer_4', min_value=32, max_value=512, step=32)
    hp_layer_5 = hp.Int('layer_5', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(units=hp_layer_4, activation=hp_activation))
    model.add(Dense(units=hp_layer_5, activation=hp_activation))
    model.add(Dense(3, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])

    return model

In [30]:
tuner_7l = kt.Hyperband(
    model_7l_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_5'
)
tuner_7l.search(x_train, y_train, epochs=10, callbacks=[stop_early])

Trial 30 Complete [00h 00m 14s]
accuracy: 0.9617373943328857

Best accuracy So Far: 0.9996055364608765
Total elapsed time: 00h 06m 42s


In [31]:
model_7l = get_best_model(tuner_7l)
model_7l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 68 units, func:  <function relu at 0x7cf30dbff2e0>
Layer-2: 448 units, func:  <function relu at 0x7cf30dbff2e0>
Layer-3: 32 units, func:  <function relu at 0x7cf30dbff2e0>
Layer-4: 480 units, func:  <function relu at 0x7cf30dbff2e0>
Layer-5: 480 units, func:  <function relu at 0x7cf30dbff2e0>
Layer-6: 128 units, func:  <function relu at 0x7cf30dbff2e0>
Layer-7: 3 units, func:  <function softmax at 0x7cf2b715e440>

Other params:
learning_rate: 0.0001
Epoch 1/100
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.9180 - loss: 0.2038 - val_accuracy: 0.9975 - val_loss: 0.0103
Epoch 2/100
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9974 - loss: 0.0108 - val_accuracy: 0.9975 - val_loss: 0.0081
Epoch 3/100
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9978 - loss: 0.0088 - val_accuracy: 0.9982 - val_loss: 0.0047


In [32]:
final_models["7_layers"] = model_7l

In [33]:
for name, model in final_models.items():
    print(f"{name}: ", end="")
    describe_model(model)
    print()

3_layers: Describe models architecture
Layer-1: 68 units, func:  <function relu at 0x7cf30dbff2e0>
Layer-2: 352 units, func:  <function tanh at 0x7cf2b715e830>
Layer-3: 3 units, func:  <function softmax at 0x7cf2b715e440>

5_layers: Describe models architecture
Layer-1: 68 units, func:  <function relu at 0x7cf30dbff2e0>
Layer-2: 128 units, func:  <function tanh at 0x7cf2b715e830>
Layer-3: 320 units, func:  <function tanh at 0x7cf2b715e830>
Layer-4: 320 units, func:  <function tanh at 0x7cf2b715e830>
Layer-5: 3 units, func:  <function softmax at 0x7cf2b715e440>

7_layers_with_dropout: Describe models architecture
Layer-1: 68 units, func:  <function relu at 0x7cf30dbff2e0>
Layer-2: 192 units, func:  <function tanh at 0x7cf2b715e830>
Layer-3: 0 units, func: None
Layer-4: 128 units, func:  <function tanh at 0x7cf2b715e830>
Layer-5: 0 units, func: None
Layer-6: 128 units, func:  <function tanh at 0x7cf2b715e830>
Layer-7: 3 units, func:  <function softmax at 0x7cf2b715e440>

7_layers: Descri

In [34]:
train_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(x_test, verbose=False)
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(y_test, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1, 2])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1, 2])

    train_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

train_set_results.sort(key=lambda k: sum(k[3]), reverse=True)
pd.DataFrame(train_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,3_layers,"[0.999, 1.0, 1.0]","[1.0, 1.0, 0.999]","[0.999, 1.0, 0.999]","[[1915, 0, 0], [0, 1844, 0], [2, 0, 1943]]"
1,5_layers,"[0.999, 1.0, 1.0]","[1.0, 1.0, 0.999]","[0.999, 1.0, 0.999]","[[1915, 0, 0], [0, 1844, 0], [2, 0, 1943]]"
2,7_layers_with_dropout,"[0.999, 1.0, 0.998]","[0.998, 1.0, 0.999]","[0.999, 1.0, 0.999]","[[1912, 0, 3], [0, 1844, 0], [2, 0, 1943]]"
3,7_layers,"[0.998, 1.0, 0.997]","[0.997, 1.0, 0.998]","[0.997, 1.0, 0.997]","[[1909, 0, 6], [0, 1844, 0], [4, 0, 1941]]"


In [35]:
# load dataset
test_df = pd.read_csv("/content/drive/MyDrive/AI_workout_assistant/test.csv")

# Categorizing label
test_df.loc[test_df["label"] == "C", "label"] = 0
test_df.loc[test_df["label"] == "H", "label"] = 1
test_df.loc[test_df["label"] == "L", "label"] = 2

print(f'Number of rows: {test_df.shape[0]} \nNumber of columns: {test_df.shape[1]}\n')
print(f"Labels: \n{test_df['label'].value_counts()}\n")

Number of rows: 710 
Number of columns: 69

Labels: 
label
1    241
2    235
0    234
Name: count, dtype: int64



In [36]:
# Standard Scaling of features
test_x = test_df.drop("label", axis = 1)
test_x = pd.DataFrame(sc.transform(test_x))

test_y = test_df["label"]

# # Converting prediction to categorical
test_y_cat = to_categorical(test_y)

In [37]:
test_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(test_x, verbose=False)
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(test_y_cat, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1, 2])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1, 2])

    test_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

test_set_results.sort(key=lambda k: sum(k[3]), reverse=True)
pd.DataFrame(test_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,3_layers,"[0.987, 1.0, 0.996]","[1.0, 0.992, 0.991]","[0.994, 0.996, 0.994]","[[234, 0, 0], [1, 239, 1], [2, 0, 233]]"
1,7_layers,"[0.987, 0.752, 1.0]","[0.662, 0.996, 0.996]","[0.793, 0.857, 0.998]","[[155, 79, 0], [1, 240, 0], [1, 0, 234]]"
2,5_layers,"[0.987, 0.738, 1.0]","[0.637, 0.996, 0.996]","[0.774, 0.848, 0.998]","[[149, 85, 0], [1, 240, 0], [1, 0, 234]]"
3,7_layers_with_dropout,"[0.918, 0.729, 1.0]","[0.62, 0.996, 0.949]","[0.74, 0.842, 0.974]","[[145, 89, 0], [1, 240, 0], [12, 0, 223]]"


In [38]:
# Dump the best model to a pickle file
with open("/content/drive/MyDrive/AI_workout_assistant/models/Planks/plank.pkl", "wb") as f:
    pickle.dump(final_models["7_layers_with_dropout"], f)

In [39]:
# Dump final results
with open("/content/drive/MyDrive/AI_workout_assistant/models/Planks/all_dp.pkl", "wb") as f:
    pickle.dump(final_models, f)